In [4]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv("/content/drive/My Drive/BT4012 Team 8/Ethereum/feature_data.csv")

In [ ]:
df

,node,isp,closeness_centrality,betweenness_centrality,eigenvector_centrality,active_days,eccentricity,pagerank_std_last_month,in_out_degree_ratio,weightsin_weightsout_ratio,numin_numout_ratio,centrality_sum,rolling_average_pagerank,cumulative_interaction_count,pagerank_change,pagerank_closeness_interaction
0,0x1f1e784a61a8ca0a90250bcd2170696655b28a21,0,0.132361,2.290699e-04,2.496896e-03,107.0,7.0,0.000000e+00,0.132389,1.000000,1.0,1.350873e-01,0.000000e+00,1327.0,0.000000e+00,8.164333e-08
1,0x1266f8b9e4dffc9e2f719bf51713f7e714516861,0,0.109312,1.747367e-07,1.795760e-05,1.0,8.0,0.000000e+00,1.000000,1.000000,1.0,1.093298e-01,0.000000e+00,1328.0,-4.834848e-07,1.457527e-08
2,0xbbfaf27674c2eb5d13edc58a40081248d13dcfeb,1,0.117714,7.214720e-05,7.059330e-05,0.0,7.0,0.000000e+00,1.000000,1.000000,0.0,1.178565e-01,5.317118e-07,0.0,7.116401e-07,9.946543e-08
3,0xb50d0c4cb2c29cc232c96a59e9c65eb82914ec75,0,0.110925,1.036793e-04,2.117960e-03,89.0,7.0,0.000000e+00,0.379518,0.434854,1.0,1.131464e-01,6.202811e-07,1445.0,7.495447e-07,9.791388e-08
4,0x563b377a956c80d77a7c613a9343699ad6123911,0,0.110763,5.919153e-03,6.486499e-03,346.0,7.0,0.000000e+00,0.010035,0.000029,1.0,1.231683e-01,4.186281e-07,9354.0,-7.983363e-07,1.538709e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29456,0xdcf880ba3487a4e95259d79994226b1e4f8d9458,0,0.000000,0.000000e+00,1.285257e-05,1.0,8.0,1.037354e-07,0.666667,1.000000,1.0,1.285257e-05,3.237338e-07,5333250.0,5.717189e-07,0.000000e+00
29457,0x2ff58a66bf3073e4c6016e1a5d8d8eb6483f5723,0,0.120039,0.000000e+00,4.736295e-04,1.0,8.0,0.000000e+00,1.000000,0.288970,0.5,1.205131e-01,1.331627e-07,5337613.0,5.578834e-12,1.598523e-08
29458,0x0d8415be40925d965ade2cd62be5680389d640e4,0,0.110962,6.770483e-06,3.166462e-05,1.0,8.0,2.400348e-08,0.500000,0.504685,1.0,1.110003e-01,1.985535e-07,5338232.0,-8.968140e-08,1.539773e-08
29459,0xe4edc0ee94ba28ac89d7304c72fa95081a3bef72,0,0.086852,2.634004e-07,1.122154e-04,2.0,8.0,3.365868e-08,0.500000,0.981012,1.0,8.696465e-02,1.845676e-07,5338238.0,-1.316392e-07,1.254592e-08


In [6]:
# Baseline -- use all default params
X = df.drop(['isp', 'node'], axis=1)
y = df['isp']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=15,shuffle=False)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

clf = SVC(probability=True)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)
class_report = classification_report(y_test, y_pred)
print("\nClassification Report:")
print(class_report)
# roc-auc
y_pred_prob = clf.predict_proba(X_test)[:, 1]
auc_test = roc_auc_score(y_test, y_pred_prob)
print(f"ROC-AUC score is {auc_test}")

Accuracy: 0.9860843986876343
Confusion Matrix:
[[8710    1]
 [ 122    6]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      8711
           1       0.86      0.05      0.09       128

    accuracy                           0.99      8839
   macro avg       0.92      0.52      0.54      8839
weighted avg       0.98      0.99      0.98      8839

ROC-AUC score is 0.8260173918034669


In [7]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Splitting the dataset into training, validation, and test sets
X = df.drop(['isp', 'node'], axis=1)
y = df['isp']
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=15, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=15) # 0.25 x 0.8 = 0.2

# Standardizing the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [ ]:
# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid']
}

# Perform Grid Search with Cross-Validation
grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=2, cv=5)
grid.fit(X_train, y_train)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...............................C=0.1, kernel=linear; total time=   1.9s
[CV] END ...............................C=0.1, kernel=linear; total time=   1.8s
[CV] END ...............................C=0.1, kernel=linear; total time=   1.6s
[CV] END ...............................C=0.1, kernel=linear; total time=   2.3s
[CV] END ...............................C=0.1, kernel=linear; total time=   4.5s
[CV] END ..................................C=0.1, kernel=rbf; total time=   1.8s
[CV] END ..................................C=0.1, kernel=rbf; total time=   1.6s
[CV] END ..................................C=0.1, kernel=rbf; total time=   1.4s
[CV] END ..................................C=0.1, kernel=rbf; total time=   1.5s
[CV] END ..................................C=0.1, kernel=rbf; total time=   1.5s
[CV] END .................................C=0.1, kernel=poly; total time=   1.3s
[CV] END .................................C=0.1,

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10],
                         'kernel': ['linear', 'rbf', 'poly', 'sigmoid']},
             verbose=2)

In [ ]:
# Best parameters and score
print("Best Parameters:", grid.best_params_)
print("Best Cross-Validation Score:", grid.best_score_)

Best Parameters: {'C': 10, 'kernel': 'rbf'}
Best Cross-Validation Score: 0.9765784623064763


In [8]:
# Evaluate on the validation set
grid = SVC(C= 10, kernel= 'rbf', probability=True)
grid.fit(X_train, y_train)

y_val_pred = grid.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy: {val_accuracy}")

# Final evaluation on the test set
y_test_pred = grid.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy: {test_accuracy}")

# Confusion Matrix and Classification Report
conf_matrix = confusion_matrix(y_test, y_test_pred)
print("Confusion Matrix:")
print(conf_matrix)
class_report = classification_report(y_test, y_test_pred)
print("\nClassification Report:")
print(class_report)
# roc-auc
y_pred_prob = grid.predict_proba(X_test)[:, 1]
auc_test = roc_auc_score(y_test, y_pred_prob)
print(f"ROC-AUC score is {auc_test}")

Validation Accuracy: 0.9804820095044128
Test Accuracy: 0.988460885796708
Confusion Matrix:
[[5818    4]
 [  64    7]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5822
           1       0.64      0.10      0.17        71

    accuracy                           0.99      5893
   macro avg       0.81      0.55      0.58      5893
weighted avg       0.98      0.99      0.98      5893

ROC-AUC score is 0.8576743870989593


## Use UnderSampling

In [9]:
# Use UnderSampling
from imblearn.under_sampling import NearMiss

X = df.drop(['isp', 'node'], axis=1)
y = df['isp']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=15,shuffle=False)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

undersample = NearMiss(version=3, n_neighbors=10)
X_under, y_under = undersample.fit_resample(X_train, y_train)
clf_under = SVC(C= 10, kernel= 'rbf', probability=True)
clf_under.fit(X_under, y_under)
print(classification_report(y_test, clf_under.predict(X_test)))

# roc-auc
y_pred_prob = clf_under.predict_proba(X_test)[:, 1]
auc_test = roc_auc_score(y_test, y_pred_prob)
print(f"ROC-AUC score is {auc_test}")

              precision    recall  f1-score   support

           0       0.99      0.81      0.89      8711
           1       0.05      0.62      0.09       128

    accuracy                           0.81      8839
   macro avg       0.52      0.71      0.49      8839
weighted avg       0.98      0.81      0.88      8839

ROC-AUC score is 0.8092740141774768


## Use Oversampling

In [10]:
# Use OverSampling
from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler(sampling_strategy='minority')
X_over, y_over = oversample.fit_resample(X_train, y_train)
clf_over = SVC(C= 10, kernel= 'rbf', probability=True)
clf_over.fit(X_over, y_over)
print(classification_report(y_test, clf_over.predict(X_test)))

# roc-auc
y_pred_prob = clf_over.predict_proba(X_test)[:, 1]
auc_test = roc_auc_score(y_test, y_pred_prob)
print(f"ROC-AUC score is {auc_test}")

              precision    recall  f1-score   support

           0       1.00      0.93      0.96      8711
           1       0.13      0.72      0.23       128

    accuracy                           0.93      8839
   macro avg       0.56      0.83      0.59      8839
weighted avg       0.98      0.93      0.95      8839

ROC-AUC score is 0.9188212102514062


## Without Scaler

In [ ]:
# Without Scaler

X = df.drop(['isp', 'node'], axis=1)
y = df['isp']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=15,shuffle=False)
clf_test = SVC(C= 10, kernel= 'rbf', probability=True)
clf_test.fit(X_train, y_train)
y_pred = clf_test.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)
class_report = classification_report(y_test, y_pred)
print("\nClassification Report:")
print(class_report)

Accuracy: 0.9855187238375381
Confusion Matrix:
[[8711    0]
 [ 128    0]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      8711
           1       0.00      0.00      0.00       128

    accuracy                           0.99      8839
   macro avg       0.49      0.50      0.50      8839
weighted avg       0.97      0.99      0.98      8839



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Use UnderSampling
from imblearn.under_sampling import NearMiss

X = df.drop(['isp', 'node'], axis=1)
y = df['isp']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=15,shuffle=False)

undersample = NearMiss(version=3, n_neighbors=10)
X_under, y_under = undersample.fit_resample(X_train, y_train)
clf_under = SVC(C= 10, kernel= 'rbf')
clf_under.fit(X_under, y_under)
print(classification_report(y_test, clf_under.predict(X_test)))

              precision    recall  f1-score   support

           0       0.99      0.98      0.98      8711
           1       0.10      0.14      0.12       128

    accuracy                           0.97      8839
   macro avg       0.54      0.56      0.55      8839
weighted avg       0.97      0.97      0.97      8839



In [ ]:
# Use OverSampling
from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler(sampling_strategy='minority')
X_over, y_over = oversample.fit_resample(X_train, y_train)
clf_over = SVC(C= 10, kernel= 'rbf')
clf_over.fit(X_over, y_over)
print(classification_report(y_test, clf_over.predict(X_test)))

              precision    recall  f1-score   support

           0       0.99      0.98      0.98      8711
           1       0.10      0.14      0.12       128

    accuracy                           0.97      8839
   macro avg       0.54      0.56      0.55      8839
weighted avg       0.97      0.97      0.97      8839

